In [182]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import os
import re
import numpy as np
from copy import deepcopy
import random
import scipy.sparse as sp
from time import time

# directory = '../MF/ml-1m'
# ratings = []
# with open(os.path.join(directory, 'ratings.dat'), encoding='latin1') as f:
#     for l in f:
#         user_id, movie_id, rating, timestamp = [int(_) for _ in l.split('::')]
#         ratings.append({
#             'user_id': user_id,
#             'movie_id': movie_id,
#             'rating': rating,
#             'timestamp': timestamp,
#             })
# ratings = pd.DataFrame(ratings)

# ratings = ratings.drop(['timestamp'],axis=1)
# print(ratings)
# # 어떤 비율로 표본 추출을 하고싶은지 df.sample에서 frac을 0~1 사이로 설정
# df = ratings.sample(frac=1).reset_index(drop=True)
# filter_user = deepcopy(df)
# counts = filter_user['user_id'].value_counts()
# filter_user = filter_user[filter_user['user_id'].isin(counts[counts >= 10].index)]

# filtered_df = deepcopy(filter_user)
# counts = filtered_df['movie_id'].value_counts()
# filtered_df = filtered_df[filtered_df['movie_id'].isin(counts[counts >=10].index)]
# filtered_df = filtered_df.reset_index(drop=True)
# print(filtered_df)


class generate_graph(object):
    def __init__(self, path):
        self.path = path  # '../NGCF-PyTorch/Data/ml-1m'
        train_file = path + '/train.txt'
        
        self.neighbor_dict = {}
        self.user, self.item = [], []
        
        with open(train_file, 'r') as f:
            for l in f.readlines():
                if len(l) > 0:
                    l = l.strip('\n').split(' ')
                    self.neighbor_dict[int(l[0])] = [int(i) for i in l[1:]]
                    self.user.append(int(l[0]))
        self.user = self.user[:1000]
        
    def jaccard_index(self, u_i, u_j, neighbor_dict):
        u_i_neighbor = self.neighbor_dict[u_i]
        u_j_neighbor = self.neighbor_dict[u_j]
        return len(list(set(u_i_neighbor) & set(u_j_neighbor))) / len(list(set(u_i_neighbor) | set(u_j_neighbor)))

    # 새로운 graph의 node j 에다가 기존 graph의 어떤 node의 neighborhood를 복사할지 zeta에 담기
    def node_copying(self):
        t1 = time()
        zeta = []
        
        for u_j in self.user:
            nor = 0
            zeta_distribution = []
            for u_i in self.user:
                nor+=self.jaccard_index(u_j, u_i, self.neighbor_dict)
            for u_m in self.user:
                zeta_distribution.append(self.jaccard_index(u_j, u_m, self.neighbor_dict) / nor)
            zeta.append(random.choices(self.user, weights=zeta_distribution)[0])
        print('total node copying time cost : ', time() - t1)
        return zeta
    
    def generate_graph(self, epsilon, iteration):
        t2 = time()
        self.epsilon = epsilon
        self.iteration = iteration
        
        generated_node = []
        
        with open(self.path+'/sampled_graph/sampled_graph_'+str(iteration+1), 'w') as f:
            for i in self.user:
                if random.uniform(0,1) < 1-self.epsilon:  # 1-epsilon의 확률로 원래 neighbor 넣기
                    generated_node.append(i)
                else:                                # epsilon의 확률로 zeta에 있는 node의 neighbor로 copy해서 넣기
                    generated_node.append(zeta[i])
                    
                # 만들어지는 새로운 graph를 txt 파일로 저장
                f.write(str(i))
                f.write(' ')
                for j in self.neighbor_dict[generated_node[i]][:-1]:
                    f.write(str(j))
                    f.write(' ')
                f.write(str(self.neighbor_dict[generated_node[i]][-1]))
                f.write('\n')
        print(iteration+1,'-th Graph sampled time cost : ', time() - t2)

In [183]:
t = generate_graph('ml-1m')
zeta = t.node_copying()

zeta
# zeta = node_copying(user)

total node copying time cost :  33.78094291687012


[706,
 324,
 468,
 630,
 491,
 994,
 929,
 290,
 827,
 845,
 402,
 264,
 43,
 623,
 918,
 227,
 441,
 823,
 614,
 864,
 213,
 842,
 828,
 41,
 596,
 763,
 477,
 216,
 785,
 961,
 142,
 967,
 219,
 652,
 936,
 801,
 790,
 790,
 462,
 296,
 693,
 91,
 375,
 222,
 53,
 532,
 836,
 9,
 6,
 406,
 963,
 801,
 876,
 930,
 477,
 153,
 615,
 492,
 108,
 894,
 122,
 756,
 230,
 776,
 984,
 893,
 863,
 470,
 280,
 790,
 599,
 245,
 393,
 458,
 117,
 8,
 722,
 574,
 95,
 288,
 822,
 455,
 838,
 425,
 84,
 85,
 203,
 691,
 108,
 127,
 219,
 497,
 600,
 946,
 78,
 845,
 831,
 49,
 329,
 925,
 764,
 155,
 91,
 493,
 474,
 102,
 176,
 425,
 910,
 422,
 715,
 925,
 601,
 932,
 97,
 815,
 60,
 636,
 453,
 132,
 646,
 380,
 284,
 123,
 773,
 123,
 286,
 843,
 47,
 852,
 915,
 994,
 333,
 423,
 804,
 385,
 504,
 882,
 314,
 270,
 49,
 898,
 354,
 571,
 265,
 969,
 929,
 456,
 778,
 456,
 920,
 276,
 273,
 955,
 95,
 395,
 51,
 204,
 524,
 746,
 587,
 14,
 644,
 795,
 345,
 237,
 206,
 80,
 557,
 29,
 476,

In [184]:
# zeta를 한번 만든 후 iteration마다 generate graph하는 방식

for epoch in range(3):
    t.generate_graph(epsilon = 0.01, iteration = epoch)

1 -th Graph sampled time cost :  0.02316117286682129
2 -th Graph sampled time cost :  0.022611618041992188
3 -th Graph sampled time cost :  0.025269031524658203


In [185]:
# GAT
# 만들어지는 graph마다 embedding 초기화 
# embedding 학습하는 GNN 구조 짜기
# 학습된 embedding 가지고 x_hat (eq.11) 구하고 BPR-OPT를 maximization 시키는 방향으로 학습

In [186]:
# data로 sparse matrix 및 adjacency matrix 만들기

class Data(object):
    def __init__(self, path):
        self.path = path
        
        train_file = path + '/train.txt'
        # path : ml-1m
        
        self.n_users, self.n_items = 0, 0
        self.exist_users = []
        with open(train_file, 'r') as f:
            for l in f.readlines():
                if len(l) > 0:
                    l = l.strip('\n').split(' ')
                    items = [int(i) for i in l[1:]]
                    uid = int(l[0])
                    self.exist_users.append(uid)
                    self.n_users = max(self.n_users, uid)
                    self.n_items = max(self.n_items, max(items))

        self.n_users+=1
        self.n_items+=1
        print(self.n_users, self.n_items)

        self.R = sp.dok_matrix((self.n_users, self.n_items), dtype=np.float32)
        self.train_items = {}

        with open(train_file, 'r') as f:
            for l in f.readlines():
                if len(l)==0:
                    break
                l = l.strip('\n').split(' ')
                uid, items = int(l[0]), [int(i) for i in l[1:]]

                for i in items:
                    self.R[uid, i] = 1

                self.train_items[uid] = items
                
    def get_adj_mat(self):
        try:
            adj_mat = sp.load_npz(self.path + '/train_adj_mat.npz')
            
        except Exception:
            adj_mat = self.create_adj_mat()
            sp.save_npz(self.path + '/train_adj_mat.npz', adj_mat)
            
        return adj_mat
    
    
    def create_adj_mat(self):
        adj_mat = self.R.todok()
        print('already create adjacency matrix', adj_mat.shape)
        return adj_mat.tocsr()


In [187]:
# node copying으로 만들어진 graph들에 대해서 adj matrix 만들고 npz 저장하는 과정 

class sampled_graph_to_matrix(object):
    def __init__(self, path, iteration):
        self.path =path
        self.iteration = iteration
        
        sampled_graph = path + '/sampled_graph/sampled_graph_' + str(iteration+1)
        # path : 'ml-1m'
        
        self.n_users, self.n_items = 0, 0
        self.exist_users = []
        with open(sampled_graph, 'r') as f:
            for l in f.readlines():
                if len(l) > 0:
                    l = l.strip('\n').split(' ')
                    items = [int(i) for i in l[1:]]
                    uid = int(l[0])
                    self.exist_users.append(uid)
                    self.n_users = max(self.n_users, uid)
                    self.n_items = max(self.n_items, max(items))

        self.n_users+=1
        self.n_items+=1
        print(self.n_users, self.n_items)

        self.R = sp.dok_matrix((self.n_users, self.n_items), dtype=np.float32)
        self.train_items = {}

        with open(sampled_graph, 'r') as f:
            for l in f.readlines():
                if len(l)==0:
                    break
                l = l.strip('\n').split(' ')
                uid, items = int(l[0]), [int(i) for i in l[1:]]

                for i in items:
                    self.R[uid, i] = 1

                self.train_items[uid] = items
                
    def get_adj_mat(self):
        try:
            adj_mat = sp.load_npz(self.path + '/s_adj_mat_' + str(self.iteration+1) + '.npz')
            
        except Exception:
            adj_mat = self.create_adj_mat()
            sp.save_npz(self.path + '/s_adj_mat_' + str(self.iteration+1) + '.npz', adj_mat)
            
        return adj_mat
    
     
    def create_adj_mat(self):
        adj_mat = self.R.todok()
        print('already create adjacency matrix', adj_mat.shape)
        return adj_mat.tocsr()


In [188]:
for iteration in range(3):
    sampled_graph = sampled_graph_to_matrix(path='ml-1m', iteration = iteration)
    sampled_graph.get_adj_mat()

1000 3953
1000 3953
1000 3953


#### ngcf에서의 w_gc와 w_bi는 gc는 neighbor aggregate하는 부분의 weight, bi는 element-wise product하는 부분의 weight
#### bgcf에서는 single feed forward layer로 구성하였고 총 3개의 weight가 학습됨 (GAT에서 2개)

In [271]:
class GATLayer(nn.Module):
    def __init__(self, n_users, n_items, in_features, out_features, dropout = 0.2):
        super(GATLayer, self).__init__()
        self.n_users = n_users
        self.n_items = n_items
        self.dropout = dropout
        self.in_features = in_features
        self.out_features = out_features
        
        initializer = nn.init.xavier_uniform_
        
        # self.W = nn.Parameter(initializer(torch.empty(in_features, out_features)))
        # W_1은 a*e aggregate 부분, W_2는 e aggregate 부분
        self.weight_dict = nn.ParameterDict({
            'W_1' : nn.Parameter(initializer(torch.empty(self.in_features, self.out_features))),
            'W_2' : nn.Parameter(initializer(torch.empty(self.in_features, self.out_features)))
        })
        
        self.embedding_dict = nn.ParameterDict({
            'user_emb' : nn.Parameter(initializer(torch.empty(self.n_users, self.out_features))),
            'item_emb' : nn.Parameter(initializer(torch.empty(self.n_items, self.out_features)))
        })
        
    
    def forward(self, path, self_feature, adj_matrix, iteration):
        self.path = path
        self.adj_matrix = adj_matrix
        coef = torch.zeros(self.n_users, self.n_items)
        
#         adj_matrix = sp.load_npz(path + '/s_adj_mat_' + str(iteration+1) +'.npz').toarray()
#         adj_matrix = torch.Tensor(adj)
        
        user_emb = self.embedding_dict['user_emb']
        item_emb = self.embedding_dict['item_emb']
        
        # self.W의 크기를 정확하게 확인해야함, 최종적인 각 node의 embedding size 는 1 x emb_size 로 나오게끔
        score = torch.exp(torch.inner(user_emb, item_emb))
        score = torch.multiply(score, adj_matrix)
        for i in range(score.shape[0]):
            norm = sum(score[i])
            score = score[i] / norm
            coef[i] = score
        w_1 = self.weight_dict['W_1']
        w_2 = self.weight_dict['W_2']
        h_tilde_1_user = torch.matmul(coef, item_emb)
        h_tilde_1_user = torch.matmul(h_tilde_1_user, w_1)
        h_tilde_1_item = torch.matmul(coef.T, user_emb)
        h_tilde_1_item = torch.matmul(h_tilde_1_item, w_1)
        h_tilde_1 = torch.cat((h_tilde_1_user, h_tilde_1_item), dim=0)
        
        neighbor_num_user, neighbor_num_item = torch.zeros(self.n_users), torch.zeros(self.n_items)
        for i in range(self.n_users):
            neighbor_num_user[i] = sum(adj_matrix[i])
        for j in range(self.n_items):
            neighbor_num_item[i] = sum(adj_matrix.T[i])
            
        h_tilde_2_user = torch.matmul(adj_matrix, item_emb)
        h_tilde_2_user = torch.matmul(neighbor_num_user, h_tilde_2_user)
        h_tilde_2_user = torch.matmul(h_tilde_2_user, w_2)
        h_tilde_2_item = torch.matmul(adj_matrix.T, user_emb)
        h_tilde_2_item = torch.matmul(neighbor_num_item, h_tilde_2_item)
        h_tilde_2_item = torch.matmul(h_tilde_2_item, w_2)
        
        print(h_tilde_1.shape, h_tilde_2.shape)
        
        h_tilde_sampled = torch.concat(h_tilde_1, h_tilde_2)
        
        return h_tilde_sampled

In [274]:
model = GATLayer(1000, 3953, 64, 64)

optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

model.train()

GATLayer(
  (weight_dict): ParameterDict(
      (W_1): Parameter containing: [torch.FloatTensor of size 64x64]
      (W_2): Parameter containing: [torch.FloatTensor of size 64x64]
  )
  (embedding_dict): ParameterDict(
      (item_emb): Parameter containing: [torch.FloatTensor of size 3953x64]
      (user_emb): Parameter containing: [torch.FloatTensor of size 1000x64]
  )
)

In [223]:
adj = sp.load_npz('ml-1m/s_adj_mat_1.npz')
adj = adj.toarray()
print(adj.shape)
adj = torch.Tensor(adj)
coef = torch.zeros(3,3953)
user_emb = nn.Parameter(initializer(torch.empty(1000,32)))
item_emb = nn.Parameter(initializer(torch.empty(3953,32)))
w_1 = nn.Parameter(initializer(torch.empty(32,32)))
w_2 = nn.Parameter(initializer(torch.empty(32,32)))

x = torch.exp(torch.inner(user_emb, item_emb))
print(x)
x = torch.multiply(x, adj)
print(x)
for i in range(3):
    tot = sum(x[i])
    score = x[i] / tot
    coef[i] = score
    
w1 = torch.matmul(coef, item_emb)
print(w1.shape)
w1 = torch.matmul(w1, w_1)
print(w1.shape)
w2 = torch.matmul(coef.T, user_emb[:3])
print(w2.shape)
w2 = torch.matmul(w2, w_2)
print(w2.shape)
w3 = torch.cat((w1, w2), dim=0)
w3.shape

user_emb = nn.Parameter(initializer(torch.empty(1000,32)))
item_emb = nn.Parameter(initializer(torch.empty(3953,32)))
avg_emb = torch.matmul(adj.T, user_emb)
avg_emb.shape
# x = torch.multiply(x, adj)
# print(x.shape)
# x = F.softmax(x, dim=1)
# x


(1000, 3953)
tensor([[1.0014, 1.0066, 1.0102,  ..., 1.0005, 1.0018, 0.9939],
        [1.0008, 1.0080, 1.0081,  ..., 1.0078, 1.0058, 0.9993],
        [1.0043, 0.9958, 0.9949,  ..., 1.0059, 0.9891, 1.0026],
        ...,
        [1.0004, 1.0004, 0.9999,  ..., 1.0017, 0.9958, 1.0016],
        [0.9911, 1.0047, 1.0027,  ..., 0.9965, 1.0048, 0.9945],
        [1.0010, 0.9965, 0.9911,  ..., 0.9995, 1.0090, 1.0023]],
       grad_fn=<ExpBackward0>)
tensor([[0.0000, 1.0066, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.9965, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)
torch.Size([3, 32])
torch.Size([3, 32])
torch.Size([3953, 32])
torch.Size([3953, 32])


torch.Size([3953, 32])

In [ ]:
def get_neighbor_num(row):
    return sum(row)

In [242]:
sumemb = torch.zeros((5))

In [249]:
for i in range(5):
    sumemb[i] = sum(adj[i])
    
torch.matmul(sumemb, item_emb[:5]).shape
# sumemb

torch.Size([32])

In [191]:
initializer = nn.init.xavier_uniform_
a = nn.Parameter(initializer(torch.empty(5,5)))
a

Parameter containing:
tensor([[ 0.3505,  0.6399,  0.2055,  0.4429, -0.6476],
        [-0.1741, -0.5893, -0.6588,  0.1689,  0.4145],
        [-0.0871,  0.2645,  0.5705, -0.1522,  0.7327],
        [ 0.4192, -0.5544,  0.3986,  0.7310,  0.5706],
        [ 0.4376,  0.7684, -0.4767,  0.1048,  0.6216]], requires_grad=True)

In [192]:
initializer = nn.init.xavier_uniform_
user_emb = nn.Parameter(initializer(torch.empty(5,10)))
user_emb

Parameter containing:
tensor([[ 0.3101,  0.3741, -0.0085,  0.4640,  0.5731, -0.2036,  0.2263,  0.3625,
          0.3871,  0.0639],
        [ 0.1042,  0.2817, -0.1724, -0.6256, -0.6170, -0.2393, -0.3559, -0.2347,
         -0.6301,  0.4292],
        [ 0.3205, -0.2222,  0.3362,  0.1307, -0.6262,  0.4004,  0.5305, -0.2964,
          0.5973, -0.4336],
        [ 0.5354, -0.0532,  0.4546,  0.4779,  0.5569, -0.4492,  0.4567, -0.4002,
         -0.6237,  0.2236],
        [-0.2962,  0.4969,  0.5059, -0.4748, -0.3745, -0.2181, -0.1347,  0.0938,
         -0.5364, -0.0535]], requires_grad=True)

In [193]:
a = np.matrix([[0,1,2],[1,1,1],[0,0,1]])
adj = np.matrix([[0,0,1],[1,0,0],[1,1,0]])
e = np.matrix([[1,2,3],[4,5,6],[7,8,9]])

print(a,'\n')
print(adj,'\n')
print(e,'\n')

a_jk = np.multiply(a,adj)
print(a_jk, '\n')

np.matmul(a_jk, e)
    
    
    

[[0 1 2]
 [1 1 1]
 [0 0 1]] 

[[0 0 1]
 [1 0 0]
 [1 1 0]] 

[[1 2 3]
 [4 5 6]
 [7 8 9]] 

[[0 0 2]
 [1 0 0]
 [0 0 0]] 



matrix([[14, 16, 18],
        [ 1,  2,  3],
        [ 0,  0,  0]])